In [40]:
import os
import warnings
import tqdm
import pandas as pd
import numpy as np
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

In [41]:
%load_ext autoreload
%autoreload 2
import socceraction.vaep.features as fs
import socceraction.vaep.labels as lab

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [42]:
datafolder = "./data-fifa/La Liga"
#0은 train, 1은 valid, 2은 test

spadl0_h5 = os.path.join(datafolder, "spadl-statsbomb_train_competitions.h5")
features0_h5 = os.path.join(datafolder, "features_train.h5")
labels0_h5 = os.path.join(datafolder, "labels_train.h5")

spadl1_h5 = os.path.join(datafolder, "spadl-statsbomb_valid_competitions.h5")
features1_h5 = os.path.join(datafolder, "features_valid.h5")
labels1_h5 = os.path.join(datafolder, "labels_valid.h5")

spadl2_h5 = os.path.join(datafolder, "spadl-statsbomb_test_competitions.h5")
features2_h5 = os.path.join(datafolder, "features_test.h5")
labels2_h5 = os.path.join(datafolder, "labels_test.h5")

In [43]:
train_games = pd.read_hdf(spadl0_h5, "games")
print("nb of games:", len(train_games))

valid_games = pd.read_hdf(spadl1_h5, "games")
print("nb of games:", len(valid_games))

test_games = pd.read_hdf(spadl2_h5, "games")
print("nb of games:", len(test_games))

nb of games: 452
nb of games: 33
nb of games: 35


In [44]:
#Select feature set X

xfns = [
    fs.actiontype,
    fs.actiontype_onehot,
    
    #foot, head, head/other, other
    #foot중 왼/오른발 구분은 bodypart에서 하지 않음
    fs.bodypart,
    fs.bodypart_onehot,
    
    #각 수행한 액션의 왼/ 오른발로 구분함
    # fs.bodypart_detailed,
    # fs.bodypart_detailed_onehot,
    
    fs.result,
    fs.result_onehot,
    
    #goalscore_team, goalscore_opponent, goalscore_diff 3개의 feature를 불러옴
    fs.goalscore,
    
    #action type and result사이의 원핫인코딩값?
    # fs.actiontype_result_onehot,
    
    
    #action의 시작, 끝 위치
    fs.startlocation,
    fs.endlocation,
    
    #movement는 x좌표변화량, y좌표변화량
    fs.movement,
    
    #space_delta는 (x,y)좌표의 변화량
    fs.space_delta,
    
    #startlocation, endloaction에서 goal까지 거리랑 각도
    fs.startpolar,
    fs.endpolar,
    
    #home인지 away인지
    fs.team,
    
    #time : 3가지 feature가 나옴
    #1.period_id = 전반1 / 후반2
    #2.time_seconds = 전/후반의 시작 후에 time
    #3.time_seconds_overall = 게임시작후 초
    # fs.time,
    
    #time_delta1 = 현 action time_seconds - 전 action time_seconds
    #time_delta2 = 현 action time_seconds - 전전 action time_seconds
    fs.time_delta,
    
]

In [45]:
nb_prev_actions = 3

Xcols = fs.feature_column_names(xfns, nb_prev_actions)
Xcols

['type_id_a0',
 'type_id_a1',
 'type_id_a2',
 'type_pass_a0',
 'type_cross_a0',
 'type_throw_in_a0',
 'type_freekick_crossed_a0',
 'type_freekick_short_a0',
 'type_corner_crossed_a0',
 'type_corner_short_a0',
 'type_take_on_a0',
 'type_foul_a0',
 'type_tackle_a0',
 'type_interception_a0',
 'type_shot_a0',
 'type_shot_penalty_a0',
 'type_shot_freekick_a0',
 'type_keeper_save_a0',
 'type_keeper_claim_a0',
 'type_keeper_punch_a0',
 'type_keeper_pick_up_a0',
 'type_clearance_a0',
 'type_bad_touch_a0',
 'type_non_action_a0',
 'type_dribble_a0',
 'type_goalkick_a0',
 'type_pass_a1',
 'type_cross_a1',
 'type_throw_in_a1',
 'type_freekick_crossed_a1',
 'type_freekick_short_a1',
 'type_corner_crossed_a1',
 'type_corner_short_a1',
 'type_take_on_a1',
 'type_foul_a1',
 'type_tackle_a1',
 'type_interception_a1',
 'type_shot_a1',
 'type_shot_penalty_a1',
 'type_shot_freekick_a1',
 'type_keeper_save_a1',
 'type_keeper_claim_a1',
 'type_keeper_punch_a1',
 'type_keeper_pick_up_a1',
 'type_clearance_a1

### train, valid, test 데이터를 X, Y데이터로 변환하는 함수

### game_id = 3773689는 score와 concede가 둘 다 true인 데이터가 존재해서 그냥 지워버림

In [46]:
def getXY(games,Xcols, features_h5, labels_h5):
    # generate the columns of the selected feature
    # X에 들어가는것이 독립변수이니까 중요한 feature가 들어가는 것은 맞지만
    # 남성 vs 여성 actions 차이
    # 시즌별 actions 차이
    # 리그별 actions 차이
    # 다양한 변수를 비교해서 actions를 비교해본다
    X = []
    for game_id in tqdm.tqdm(games.game_id, desc="Selecting features"):
        if game_id == 3773689:
            print("game_id = ",3773689,"인  game은 제거합니다")
            continue
        
        Xi = pd.read_hdf(features_h5, f"game_{game_id}")
        X.append(Xi[Xcols])
    X = pd.concat(X).reset_index(drop=True)

    # 2. Select label Y
    Ycols = ["scores","concedes"]
    Y = []
    for game_id in tqdm.tqdm(games.game_id, desc="Selecting label"):
        if game_id == 3773689:
            print("game_id = ",3773689,"인  game은 제거합니다")
            continue
        Yi = pd.read_hdf(labels_h5, f"game_{game_id}")
        Y.append(Yi[Ycols])
    Y = pd.concat(Y).reset_index(drop=True)
    
    return X, Y

In [47]:
X_train, Y_train = getXY(train_games,Xcols, features0_h5, labels0_h5)
X_train.shape, Y_train.shape

Selecting label: 100%|██████████| 452/452 [00:02<00:00, 189.82it/s]


((998780, 154), (998780, 2))

In [48]:
X_valid, Y_valid = getXY(valid_games,Xcols, features1_h5, labels1_h5)
X_valid.shape, Y_valid.shape

Selecting label: 100%|██████████| 33/33 [00:00<00:00, 182.95it/s]


((75786, 154), (75786, 2))

In [49]:
X_test, Y_test = getXY(test_games,Xcols, features2_h5, labels2_h5)
X_test.shape, Y_test.shape

Selecting features: 100%|██████████| 35/35 [00:00<00:00, 85.42it/s]


game_id =  3773689 인  game은 제거합니다


Selecting label: 100%|██████████| 35/35 [00:00<00:00, 189.29it/s]

game_id =  3773689 인  game은 제거합니다


((79812, 154), (79812, 2))

## -Data preprocessing

1. time_delta 오류값 처리 <br><br>
- X_train, X_valid, X_test에는 오류값이 들어있는데, 바로 time_delta값이다<br>
- time_delta = 현재 action time - 전 action time <br>
- 위 방식대로 각 경기를 연결하면, 현재 경기의 첫번째 time_delta = 현재 경기 첫 time - 전 경기 마지막 time가 들어가므로 오류가 발생함 => 개별적으로 전처리해줌<br>

In [50]:
count1 = 0
count2 = 0
count3 = 0

#train, valid, test모두 time_delata값에 음수가 들어가는 현상발생
for i,value in enumerate(X_train['time_delta_1']):
    if value<0:
        count1+=1
        
for i,value in enumerate(X_valid['time_delta_1']):
    if value<0:
        count2+=1
        
for i,value in enumerate(X_test['time_delta_1']):
    if value<0:
        count3+=1

count1,count2,count3

(452, 33, 34)

time_prprocessing함수는 time_delta_1, time_delta_2의 feature를 전처리하는 함수로<br>
음수로 표현된 잘못 된 값은 이전action이 없어서 나오는 값이므로 0으로 처리한다

In [51]:
def time_preprocessing(data):
    for i in tqdm.tqdm(range(len(data))):
        if data.loc[i,'time_delta_1'] < 0.0:
            data.loc[i,'time_delta_1'] = 0.0
            
        if data.loc[i,'time_delta_2'] < 0.0:
            data.loc[i,'time_delta_2'] = 0.0    

    return data

In [52]:
X_train = time_preprocessing(X_train)
X_valid = time_preprocessing(X_valid)
X_test = time_preprocessing(X_test)

100%|██████████| 79812/79812 [00:01<00:00, 68174.27it/s]


In [53]:
X_train.shape, X_valid.shape,X_test.shape

((998780, 154), (75786, 154), (79812, 154))

In [54]:
Y_train.shape, Y_valid.shape, Y_test.shape

((998780, 2), (75786, 2), (79812, 2))

2. test_data에서 game_id = 3773689는 score와 concede가 둘 다 true인 데이터가 존재해서 해당 경기 데이터 제거되었는지 확인

In [55]:
print("Y_train ",Y_train.value_counts(),'\n')
print("Y_valid ",Y_valid.value_counts(),'\n')
#test데이터에는 둘다 score=true & concede=true인 데이터는 존재X
print("Y_test ",Y_test.value_counts())

Y_train  scores  concedes
False   False       983349
True    False        13272
False   True          2159
Name: count, dtype: int64 

Y_valid  scores  concedes
False   False       74776
True    False         858
False   True          152
Name: count, dtype: int64 

Y_test  scores  concedes
False   False       78735
True    False         940
False   True          137
Name: count, dtype: int64


3. 동일 작업없이 미리 데이터 저장하기

In [56]:
X_train.to_csv('./soccer_binary_data/train/X_train')
Y_train.to_csv('./soccer_binary_data/train/Y_train')

In [57]:
X_valid.to_csv('./soccer_binary_data/valid/X_valid')
Y_valid.to_csv('./soccer_binary_data/valid/Y_valid')

In [58]:
X_test.to_csv('./soccer_binary_data/test/X_test')
Y_test.to_csv('./soccer_binary_data/test/Y_test')

4. 현재 이기고 있는 상황 / 비기고 있는 상황 / 지고 있는 상황을 categorical feature로 추가

-기존 goalscore=[-8,8]의 수치형 데이터가 존재하는데, 본 연구에서는 추가적인 feature를 사용하기 위해 goalscore를 활용하여  승/패/무의 categorical data로 추가

In [59]:
def categorical_goal(X):
    except_feature = ['goalscore_team', 'goalscore_opponent', 'goalscore_diff']
    for except_col in except_feature:
    #카테고리로 넣었을 때는, 패무승 중인 경우 3가지로 구분해서 feature를 추가
        if except_col=='goalscore_diff':
            #기존 수치형데이터인 득실차는 유지하고
            #새로운 카테고리 승/무/패인 상황을 추가함
            cate = "cate_" + except_col
            for i, value in tqdm.tqdm(enumerate(X[except_col]),desc='lose:0 equal:1 win:2'):
                if value<0:
                    X.loc[i,cate] = 0
                elif value==0:
                    X.loc[i,cate] = 1
                else:
                    X.loc[i,cate] = 2
                    
        X[except_col] = X[except_col].astype('float64')
                    
    return X

In [60]:
#승/무/패인 상황을 categorical호 집어넣는 데이터 -> new_c_train & new_c_test
#본 연구에서는 성능이 좀 더 떨어지므로 해당 feature는 사용하진 않음
# categorical_goalscore_train = categorical_goal(X_train)
# categorical_goalscore_valid = categorical_goal(X_valid)
# categorical_goalscore_test = categorical_goal(X_test)

5. zone categorical data추가

-기존 position데이터가 존재하지만, 각 위치를 구역(zone, grid)별로 만들어서 categorical data로 사용할 예정

In [61]:
def get_zone_index(x, y):
    zone_width = 108/12  # zone의 너비
    zone_height = 72/8  # zone의 높이

    # x좌표가 속하는 zone의 인덱스 계산
    zone_col = int(x // zone_width)
    if zone_col == 12:  # 경계 체크
        print("sadas")
        zone_col -= 1

    # y좌표가 속하는 zone의 인덱스 계산
    zone_row = int(y // zone_height)
    if zone_row == 8:  # 경계 체크
        zone_row -= 1

    # zone의 인덱스를 반환
    return zone_row * 12 + zone_col

In [62]:
def location_zone(X):
    position_feature = [['start_x_a0','start_y_a0'],['start_x_a1','start_y_a1'],
                        ['start_x_a2','start_y_a2'],['end_x_a0','end_y_a0'],
                        ['end_x_a1','end_y_a1'],['end_x_a2','end_y_a2']]
    
    for x_col,y_col in position_feature:
        #기존 수치형데이터인 위치데이터는 유지하고
        #새로운 zone feature을 추가함
        col_name = 'zone_' + x_col[-2:]
        for i,(x_val, y_val) in tqdm.tqdm(enumerate(zip(X[x_col],X[y_col])),desc='location discrete by zone'):
            X.loc[i,col_name] = get_zone_index(x_val,y_val)
            
    return X

In [63]:
# location_train = location_zone(X_train)
# location_valid = location_zone(X_valid)
# location_test = location_zone(X_test)

## 이진분류뿐 아니라 다중 분류를 활용하기 위해 label=0/1/2인 multi_data를 만듬

In [64]:
multi_X_train = pd.read_csv('./soccer_binary_data/train/X_train',index_col=0)
multi_Y_train = pd.read_csv("./soccer_binary_data/train/Y_train",index_col=0)           

multi_X_valid = pd.read_csv('./soccer_binary_data/valid/X_valid',index_col=0)
multi_Y_valid = pd.read_csv("./soccer_binary_data/valid/Y_valid",index_col=0)           

multi_X_test = pd.read_csv('./soccer_binary_data/test/X_test',index_col=0)
multi_Y_test = pd.read_csv("./soccer_binary_data/test/Y_test",index_col=0)  

In [65]:
multi_X_train.shape, multi_Y_train.shape, multi_X_valid.shape, multi_Y_valid.shape, multi_X_test.shape, multi_Y_test.shape

((998780, 154),
 (998780, 2),
 (75786, 154),
 (75786, 2),
 (79812, 154),
 (79812, 2))

In [66]:
def multi_class(Y):
    multi_label = pd.DataFrame(columns=['label'])
    
    for i in tqdm.tqdm(range(len(Y))):
        if (Y.loc[i,'scores']==False) & (Y.loc[i,'concedes']==False):
            multi_label.loc[i,'label'] = 0
        elif (Y.loc[i,'scores']==True) & (Y.loc[i,'concedes']==False):
            multi_label.loc[i,'label'] = 1
        elif (Y.loc[i,'scores']==False) & (Y.loc[i,'concedes']==True):
            multi_label.loc[i,'label'] = 2
        else:
            print("error : 'score=True, concede=True' is impossible")
            exit()
    return multi_label

In [ ]:
multi_Y_train = multi_class(multi_Y_train)
multi_Y_valid = multi_class(multi_Y_valid)
multi_Y_test = multi_class(multi_Y_test)

In [ ]:
multi_Y_train = multi_class(multi_Y_train)
multi_Y_valid = multi_class(multi_Y_valid)
multi_Y_test = multi_class(multi_Y_test)

In [ ]:
multi_X_train.to_csv('./soccer_multiclass_data/train/X_train')
multi_Y_train.to_csv('./soccer_multiclass_data/train/Y_train')

multi_X_valid.to_csv('./soccer_multiclass_data/valid/X_valid')
multi_Y_valid.to_csv('./soccer_multiclass_data/valid/Y_valid')

multi_X_test.to_csv('./soccer_multiclass_data/test/X_test')
multi_Y_test.to_csv('./soccer_multiclass_data/test/Y_test')